In [1]:
%pip install -qU langmem langgraph langchain_voyageai langgraph-checkpoint-mongodb langgraph-store-mongodb pymongo requests pypdf langchain-community

Note: you may need to restart the kernel to use updated packages.


In [2]:
import getpass
import os

# 安全地获取并设置环境变量的函数
def set_env_securely(var_name, prompt):
    value = getpass.getpass(prompt)
    os.environ[var_name] = value

In [8]:
from langchain_openai import AzureChatOpenAI

# 使用 Azure 时建议关闭 tracing，避免 401（与 zero_to_hero 中一致）
os.environ["OPENAI_AGENTS_DISABLE_TRACING"] = "true"

azure_llm = AzureChatOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),  # 或直接写 "https://xxx.openai.azure.com/"
    api_key=os.getenv("OPENAI_API_KEY"),
    api_version="2025-03-01-preview",  # 或 "2024-12-01-preview"
    azure_deployment="gpt-4o",          # 你的 deployment 名称
    model="gpt-4o",                     # 模型类型，与 deployment 对应
    temperature=0,
)


In [13]:
# Import core components 
from langchain_openai import AzureOpenAIEmbeddings
from langgraph.prebuilt import create_react_agent
from langgraph.store.memory import InMemoryStore
from langmem import create_manage_memory_tool, create_search_memory_tool
from langchain_voyageai import VoyageAIEmbeddings




azure_embeddings=VoyageAIEmbeddings(model="voyage-3-large"),


# Set up storage 
store = InMemoryStore(
    index={
        "dims": 1536,
        "embed": azure_embeddings,
    }
) 

# Create an agent with memory capabilities 
agent = create_react_agent(
    azure_llm,
    tools=[
        # Memory tools use LangGraph's BaseStore for persistence (4)
        create_manage_memory_tool(namespace=("memories",)),
        create_search_memory_tool(namespace=("memories",)),
    ],
    store=store,
)

/Users/binzhou/Demo/agent_memory_demo/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/var/folders/0c/6xdkz3fs2m931gvzk6gz44mh0000gp/T/ipykernel_36186/3165142422.py:23: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent = create_react_agent(


In [14]:
# Store a new memory 
agent.invoke(
    {"messages": [{"role": "user", "content": "Remember that I prefer dark mode."}]}
)

# Retrieve the stored memory 
response = agent.invoke(
    {"messages": [{"role": "user", "content": "What are my lighting preferences?"}]}
)
print(response["messages"][-1].content)
# Output: "You've told me that you prefer dark mode."

TypeError: 'tuple' object is not callable